In [4]:
import pandas as pd
import requests as rq
import json

In [5]:


# This is the user agent string of Microsoft Edge. It's to trick the website that these requests are coming from old people who can't use computers!
header_data= { "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14931"}

In [58]:
seasons=[ "2012-13","2013-14","2014-15","2015-16","2016-17","2017-18","2018-19"]
venues=["Home" , "Road" ]

# set up lists to hold reponse info
year =[]
location =[]
team =[]
win=[]
loss = []
win_pct =[]
ftm = []
fta =[]
ft_pct =[]
w_rnk =[]
l_ank =[]
w_pct_rnk = []
ftm_rnk = []
fta_rnk =[]
ft_pct_rnk =[]

#i is the index for season
i=0
#j is the index for home_away
j=1


for season in seasons:
    for venue in venues:
        urlliteral= f"https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location={venue}&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision="
        new=rq.get(urlliteral, headers=header_data).json()
        for t in range(0,30,1):
            team.append(new["resultSets"][0]["rowSet"][t][1])
            win.append(new["resultSets"][0]["rowSet"][t][3])
            loss.append(new["resultSets"][0]["rowSet"][t][4])
            win_pct.append(new["resultSets"][0]["rowSet"][t][5])
            ftm.append(new["resultSets"][0]["rowSet"][t][13])
            fta.append(new["resultSets"][0]["rowSet"][t][14])
            ft_pct.append(new["resultSets"][0]["rowSet"][t][15])
            w_rnk.append(new["resultSets"][0]["rowSet"][t][29])
            l_ank.append(new["resultSets"][0]["rowSet"][t][30])
            w_pct_rnk.append(new["resultSets"][0]["rowSet"][t][31])
            ftm_rnk.append(new["resultSets"][0]["rowSet"][t][39])
            fta_rnk.append(new["resultSets"][0]["rowSet"][t][40])
            ft_pct_rnk.append(new["resultSets"][0]["rowSet"][t][41])
            year.append(new['parameters']["Season"])
            location.append(new['parameters']["Location"])
        print(f"Getting data for {new['parameters']['Season']} {new['parameters']['Location']}")

Print("Data scrape is done")

Getting data for 2012-13 Home
Getting data for 2012-13 Road
Getting data for 2013-14 Home
Getting data for 2013-14 Road
Getting data for 2014-15 Home
Getting data for 2014-15 Road
Getting data for 2015-16 Home
Getting data for 2015-16 Road
Getting data for 2016-17 Home
Getting data for 2016-17 Road
Getting data for 2017-18 Home
Getting data for 2017-18 Road
Getting data for 2018-19 Home
Getting data for 2018-19 Road


NameError: name 'Print' is not defined

In [56]:
nba_dict  = {
   "Team": team,
    "Season": year,
   "Venue": location,
   "Wins": win,
   "Loss": loss,
   "Win%": win_pct,
   "FT Made": ftm,
   "FT Att": fta,
   "FT%": ft_pct,
   "Win Rnk": w_rnk,
   "Loss Rnk": l_ank,
   "Win% Rnk": w_pct_rnk,
   "FT Made Rnk": ftm_rnk,
   "FT ATT Rnk": fta_rnk,
   "FT% Rnk": ft_pct_rnk
}

nba_data = pd.DataFrame(nba_dict)
nba_data

,Team,Season,Venue,Wins,Loss,Win%,FT Made,FT Att,FT%,Win Rnk,Loss Rnk,Win% Rnk,FT Made Rnk,FT ATT Rnk,FT% Rnk
0,Atlanta Hawks,2012-13,Home,25,16,0.610,595,856,0.695,15,15,15,27,24,28
1,Boston Celtics,2012-13,Home,27,13,0.675,675,889,0.759,13,12,13,20,21,15
2,Brooklyn Nets,2012-13,Home,26,15,0.634,724,1003,0.722,14,14,14,11,10,25
3,Charlotte Bobcats,2012-13,Home,15,26,0.366,823,1076,0.765,28,28,28,3,6,11
4,Chicago Bulls,2012-13,Home,24,17,0.585,685,892,0.768,16,16,16,17,20,10
5,Cleveland Cavaliers,2012-13,Home,14,27,0.341,707,933,0.758,29,29,29,13,13,16
6,Dallas Mavericks,2012-13,Home,24,17,0.585,678,868,0.781,16,16,16,19,23,4
7,Denver Nuggets,2012-13,Home,38,3,0.927,766,1089,0.703,1,1,1,7,4,27
8,Detroit Pistons,2012-13,Home,18,23,0.439,638,918,0.695,25,25,25,25,15,29
9,Golden State Warriors,2012-13,Home,28,13,0.683,704,902,0.780,12,12,12,16,16,6


In [57]:
nba_data.to_csv("NBA Free Throw Data.csv")